In [62]:
import numpy as np
import pandas as pd
from daty_transakcji import *
from osoby import *
from collections import Counter
from samochody import *
from czesci import *

# Pracownicy

In [63]:
birth_dates_lims = [1960, 1995]
begining_date = "2014-01-01"
end_date = "2018-01-01"

def rind(a, b):
    return np.random.randint(a, b)

jobs_list = np.array(["menadżer", "sprzedawca", "mechanik", "pomocnik mechanika", 
                       "diagnosta samochodowy", "sprzątający", "informatyk", "księgowy"])
jobs_amount = np.array([1, rind(2, 5), rind(3, 6), rind(1, 3),
                        rind(1, 3), rind(1, 3), rind(1, 3), rind(0, 2)])
fixed_jobs = np.array([0, 1, 2, np.sum(jobs_amount[:2]), np.sum(jobs_amount[0:6])]) # zaczynamy z szefem, dwoma sprzedawcami i mechanikiem

employment_dates = random_transaction_dates(begining_date, end_date, np.sum(jobs_amount))
employment_dates[fixed_jobs] = begining_date  

wages_lims = [(60, 75), (45, 60), (45, 60), (35, 45),
              (47, 60), (35, 40), (40, 60), (40, 53)]

employees_df = pd.DataFrame(columns=["id", "imię", "nazwisko", "PESEL",
                                     "miasto", "adres", "telefon", "email", "data_zatrudnienia",
                                     "data_zwolnienia", "stanowisko", "płaca"])
dismissal_dates = [[] for i in range(len(jobs_list))]
for (i, job) in enumerate(jobs_list):

    begin_times = ["" for j in range(jobs_amount[i])]
    for j in range(jobs_amount[i]):
        employment_date = employment_dates[np.sum(jobs_amount[:i]) + j]
        if np.random.rand() <= 0.5:
            first_name, last_name = random_name_man()
            pesel, birth_date = random_pesel_date_man(birth_dates_lims)
            sex = "M"
            city = random_city(prob=0.7)
            address = random_street()
            phone = random_phone()
            #email = random_email(first_name, last_name, birth_date[-4:])
            email = employee_email(first_name, last_name)
            employment_date = employment_dates[np.sum(jobs_amount[:i]) + j]
            wage = np.random.choice(np.arange(wages_lims[i][0], wages_lims[i][1], 0.1))
        else:
            first_name, last_name = random_name_woman()
            pesel, birth_date = random_pesel_date_woman(birth_dates_lims)
            sex = "K"
            city = random_city(prob=0.7)
            address = random_street()
            phone = random_phone()
            #email = random_email(first_name, last_name, birth_date[-4:])
            email = employee_email(first_name, last_name)
            employment_date = employment_dates[np.sum(jobs_amount[:i]) + j]
            wage = np.random.choice(np.arange(wages_lims[i][0], wages_lims[i][1], 0.1))
        begin_times[j] = employment_date

        employees_df.loc[np.sum(jobs_amount[:i]) + j] = [0, first_name, last_name, pesel,
                                                         city, address, phone, email, employment_date,
                                                         None, job, wage]
    dismissal_dates_j = [None for j in range(jobs_amount[i])]
    c = 0
    for j in range(len(begin_times)):
        if len(begin_times) >= 2:
            min_date = min(begin_times[:j] + begin_times[j+1:])
        else:
            min_date = begin_times[j]
        if np.random.rand() <= (jobs_amount[i]-1)/10 * 1/(1+c):
            dismissal_date = random_transaction_dates(str(max(min_date, begin_times[j])), end_date, 1)[0]
            okay = False
            for k in range(len(dismissal_dates_j)):
                if (dismissal_dates_j[k] is not None) and (dismissal_date >=  begin_times[k]) and (dismissal_date <= dismissal_dates_j[k]):
                    okay = True
                    break
                elif dismissal_date >=  begin_times[k]:
                    okay = True
                    break
            if okay:
                dismissal_dates_j[j] = dismissal_date
                c += 1
    dismissal_dates[i] = dismissal_dates_j

diss = []
for i in range(len(dismissal_dates)):
    diss += dismissal_dates[i]
employees_df["data_zwolnienia"] = diss

employees_df = employees_df.sort_values(by="data_zatrudnienia")   
employees_df["id"] =  np.arange(1, len(employees_df)+1)

In [64]:
employees_df

,id,imię,nazwisko,PESEL,miasto,adres,telefon,email,data_zatrudnienia,data_zwolnienia,stanowisko,płaca
0,1,Robert,Czopek,61111187231,Iłowa,ul. Piotra Skargi 16,48851178913,robert.czopek@kuba.pl,2014-01-01,None,menadżer,68.6
1,2,Liudmyla,Dąbrowska,94062499861,Wrocław,ul. Glinki 22/49,48243205224,liudmyla.dabrowska@kuba.pl,2014-01-01,2014-01-10,sprzedawca,45.7
2,3,Paweł,Kaza,61091330436,Wrocław,ul. Ligonia 98/9,48357957704,pawel.kaza@kuba.pl,2014-01-01,None,sprzedawca,53.2
5,4,Paweł,Suravets,71121896553,Wrocław,ul. Kolejowa 56/13,48620840383,pawel.suravets@kuba.pl,2014-01-01,2017-10-10,mechanik,52.4
12,5,Jakub,Macioła,72042204854,Wrocław,ul. Kazimierza Jagiellończyka 10/41,48914526984,jakub.maciola@kuba.pl,2014-01-01,None,informatyk,42.7
3,6,Erwin,Pudłowski,85010803397,Wrocław,ul. św. Stanisława Kostki 7/45,48361038633,erwin.pudlowski@kuba.pl,2015-02-03,None,sprzedawca,48.4
4,7,Stefania,Berus,63020930504,Wrocław,ul. Jasna 11,48321298509,stefania.berus@kuba.pl,2015-10-04,None,sprzedawca,45.8
6,8,Ryszard,Rzeszotarski,93091495512,Wrocław,ul. Władysława IV 8/35,48527825791,ryszard.rzeszotarski@kuba.pl,2016-05-17,None,mechanik,59.6
7,9,Agata,Fangrat,71122253340,Wrocław,ul. Szkolna 60,48561599790,agata.fangrat@kuba.pl,2016-06-18,None,mechanik,49.3
8,10,Teresa,Joś,94052747567,Wrocław,ul. Grabowa 11/16,48748581454,teresa.jos@kuba.pl,2016-07-02,None,pomocnik mechanika,39.2


In [65]:
employees_df.to_csv("outputs/pracownicy.csv", index=False)

# Transakcje

In [66]:
no_transactions = np.random.poisson(2000)
begining_buy = 20

dates = random_transaction_dates(begining_date, end_date, no_transactions)

In [67]:
client_ids = np.zeros(len(dates), dtype=int) # array with client ids for table with transactions

j = 1
for i in range(len(dates)):
    prob = i/(2*len(dates))
    if np.random.rand() <= prob:
        client_ids[i] = np.random.choice(np.unique(client_ids[:i]))
    else:
        client_ids[i] = j
        j += 1

In [68]:
car_ids = np.zeros(len(dates), dtype=int) # array with car ids for table with transactions
car_ids[:begining_buy] = np.arange(1, 21)

client_ids = np.zeros(len(dates), dtype=int) # array with client ids for table with transactions
client_ids[:begining_buy] = np.arange(1, 21)
year_ceil = [int(date[:4])-19 for date in dates[:20]]

transactions_types = np.zeros(len(dates))

clients_cars = {}
our_cars = np.arange(1, 21).tolist()
not_fixed_our_cars = our_cars[:]

jp = 21
jc = 21
for i in range(20, len(dates)):
    weights = np.array([1.5*len(clients_cars)/max(1, len(our_cars)) , len(our_cars)*50, len(clients_cars)/(max(1, len(our_cars)))])
    weights = weights/np.sum(weights)
    transaction_type = np.random.choice(np.arange(3), p=weights) # 0-buy, 1-sell, 2-service
    transactions_types[i] = transaction_type
    if transaction_type == 0:
        prob = i/(2*len(dates)) 
        if np.random.rand() <= prob: #client that already is in database
            client_ids[i] = np.random.choice(np.unique(client_ids[:i]))
            if client_ids[i] in clients_cars and np.random.rand() <= 0.6 and len(clients_cars[client_ids[i]]) >= 1:
                car_ids[i] = np.random.choice(clients_cars[client_ids[i]])
                clients_cars[client_ids[i]].remove(car_ids[i])
            else:
                car_ids[i] = jc
                jc += 1
            our_cars.append(car_ids[i])
            not_fixed_our_cars.append(car_ids[i])
        else:
            client_ids[i] = jp
            car_ids[i] = jc
            our_cars.append(car_ids[i])
            not_fixed_our_cars.append(car_ids[i])
            jp += 1
            jc += 1
            year_ceil.append(int(dates[i][:4])-19)

    if transaction_type == 1:
        prob = i/(2*len(dates))
        if np.random.rand() <= prob: #client that already is in database
            client_ids[i] = np.random.choice(np.unique(client_ids[:i]))
            car_ids[i] = np.random.choice(our_cars)
            if client_ids[i] in clients_cars:
                clients_cars[client_ids[i]].append(car_ids[i])
            else:
                clients_cars[client_ids[i]] = [car_ids[i]]
            our_cars.remove(car_ids[i])
            if car_ids[i] in not_fixed_our_cars:
                not_fixed_our_cars.remove(car_ids[i])
        else:
            client_ids[i] = jp
            car_ids[i] = np.random.choice(our_cars)
            clients_cars[client_ids[i]] = [car_ids[i]]
            our_cars.remove(car_ids[i])
            if car_ids[i] in not_fixed_our_cars:
                not_fixed_our_cars.remove(car_ids[i])
            jp += 1
            year_ceil.append(int(dates[i][:4])-19)
    
    if transaction_type == 2:
        prob = i/(2*len(dates))
        if len(not_fixed_our_cars) >= 1 and np.random.rand() <= weights[1]/2: #fixing car we currently have
           client_ids[i] = 0
           car_ids[i] = np.random.choice(not_fixed_our_cars)
           not_fixed_our_cars.remove(car_ids[i])
        elif np.random.rand() <= prob: #client that already is in database
            client_ids[i] = np.random.choice(np.unique(client_ids[:i]))
            if client_ids[i] in clients_cars:
                if np.random.rand() <= 0.6 and len(clients_cars[client_ids[i]]) >= 1:
                    car_ids[i] = np.random.choice(clients_cars[client_ids[i]])
                else:
                    car_ids[i] = jc
                    clients_cars[client_ids[i]].append(car_ids[i])
                    jc += 1
            else:
                car_ids[i] = jc
                clients_cars[client_ids[i]] = [car_ids[i]]
                jc += 1
        else:
            client_ids[i] = jp
            car_ids[i] = jc
            clients_cars[client_ids[i]] = [car_ids[i]]
            jp += 1
            jc += 1
            year_ceil.append(int(dates[i][:4])-19)

In [69]:
len(year_ceil), len(pd.unique(client_ids))

(1432, 1433)

In [70]:
year_ceil

[1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,

In [71]:
buy_indexes = np.where(transactions_types == 0)
sell_indexes = np.where(transactions_types == 1)
service_indexes = np.where(transactions_types == 2)

In [72]:
len(buy_indexes[0]), len(sell_indexes[0]), len(service_indexes[0])

(752, 748, 524)

In [73]:
transaction_dates_buy = dates[buy_indexes]
transaction_dates_sell = dates[sell_indexes]
transaction_dates_service = dates[service_indexes]

client_ids_buy = client_ids[buy_indexes]
client_ids_sell = client_ids[sell_indexes]
client_ids_service = client_ids[service_indexes]

car_ids_buy = car_ids[buy_indexes]
car_ids_sell = car_ids[sell_indexes]
car_ids_service = car_ids[service_indexes]

In [74]:
buy_employee_id = np.zeros(len(transaction_dates_buy))

for i in range(len(transaction_dates_buy)):
    filter1 = employees_df["stanowisko"] == "sprzedawca"
    filter2 = employees_df["data_zatrudnienia"] <= transaction_dates_buy[i]
    filter3 = pd.notna(employees_df["data_zwolnienia"])
    filter4 = employees_df["data_zwolnienia"] < transaction_dates_buy[i]
    buy_employee_id[i] = np.random.choice(employees_df['id'][filter1 & filter2 & ~(filter3 & filter4)])

In [75]:
sell_employee_id = np.zeros(len(transaction_dates_sell))

for i in range(len(transaction_dates_sell)):
    filter1 = employees_df["stanowisko"] == "sprzedawca"
    filter2 = employees_df["data_zatrudnienia"] <= transaction_dates_sell[i]
    filter3 = pd.notna(employees_df["data_zwolnienia"])
    filter4 = employees_df["data_zwolnienia"] < transaction_dates_sell[i]
    sell_employee_id[i] = np.random.choice(employees_df['id'][filter1 & filter2 & ~(filter3 & filter4)])

In [76]:
service_employee_id = np.zeros(len(transaction_dates_service))

for i in range(len(transaction_dates_service)):
    filter1 = employees_df["stanowisko"] == "mechanik"
    filter2 = employees_df["data_zatrudnienia"] <= transaction_dates_service[i]
    filter3 = pd.notna(employees_df["data_zwolnienia"])
    filter4 = employees_df["data_zwolnienia"] < transaction_dates_service[i]
    service_employee_id[i] = np.random.choice(employees_df['id'][filter1 & filter2 & ~(filter3 & filter4)])


# klienci

In [77]:
year_floor = 1960
transaction_counter = Counter(client_ids)

clients_df = pd.DataFrame(columns=["id", "imię", "nazwisko", "płeć", "PESEL", "miasto", "adres", "karta_lojalnościowa", "telefon", "email"])

menager = employees_df.iloc[0]
if int(menager["PESEL"][-2]) % 2 == 1:
    menager_sex = "M"
else:
    menager_sex = "K"
clients_df.loc[0] = [0, menager["imię"], menager["nazwisko"], menager_sex, menager["PESEL"], menager["miasto"], menager["adres"], False, menager["telefon"], menager["email"]]

unique_clients = np.unique(client_ids)
unique_clients = unique_clients[unique_clients != 0]
for i in range(len(unique_clients)):
    if np.random.rand() <= 0.5: # chłop probability
        id_ = unique_clients[i]
        first_name, last_name = random_name_man()
        if id_ in client_ids_buy or id_ in client_ids_sell:
            pesel, birth_date = random_pesel_date_man([year_floor, year_ceil[i]])
            year = birth_date[-4:]
        else:
            pesel = None
            birth_date = None
            year = str(np.random.randint(year_floor, year_ceil[i]))
        sex = 'M'
        city = random_city()
        address = random_street()
        if transaction_counter[id_] >= 3:
            loyalty_card = True
        else:
            loyalty_card = False
        phone = random_phone()
        email = random_email(first_name, last_name, year)
        clients_df.loc[i+1] = [unique_clients[i], first_name, last_name, sex, pesel, city, address, loyalty_card, phone, email]
    else:
        id_ = unique_clients[i]
        first_name, last_name = random_name_woman()
        if id_ in client_ids_buy or id_ in client_ids_sell:
            pesel, birth_date = random_pesel_date_man([year_floor, year_ceil[i]])
            year = birth_date[-4:]
        else:
            pesel = None
            birth_date = None
            year = str(np.random.randint(year_floor, year_ceil[i]))
        sex = 'K'
        city = random_city()
        address = random_street()
        if transaction_counter[id_] >= 3:
            loyalty_card = True
        else:
            loyalty_card = False
        phone = random_phone()
        email = random_email(first_name, last_name, year)
        clients_df.loc[i+1] = [unique_clients[i], first_name, last_name, sex, pesel, city, address, loyalty_card, phone, email]
                             
clients_df.astype({"PESEL": str})

,id,imię,nazwisko,płeć,PESEL,miasto,adres,karta_lojalnościowa,telefon,email
0,0,Robert,Czopek,M,61111187231,Iłowa,ul. Piotra Skargi 16,False,48851178913,robert.czopek@kuba.pl
1,1,Lena,Skwarczek,K,60062446615,Wrocław,ul. Michała Poleskiego 28/47,False,48448167425,u01q1a8q1r2x0m52@opayq.com
2,2,Ewelina,Jasińska,K,76081037596,Drawno,ul. Bolesława Krzywoustego 70/30,False,48423071263,ejasinska76@gmail.com
3,3,Dawid,Lisewski,M,60092784275,Wrocław,ul. Pocztowa 60/15,False,48149667875,dlisewski60@gmail.com
4,4,Danuta,Pawlik,K,94051955617,Wrocław,ul. Wiosenna 27,False,48783613153,dan-paw@gmail.com
...,...,...,...,...,...,...,...,...,...,...
1428,1428,Ewa,Lutostańska,K,69011606838,Wrocław,ul. Ks. Jerzego Popiełuszki 69,False,48635885839,l3y6ksv3@opayq.com
1429,1429,Krzysztof,Lempaszak,M,97020931111,Piastów,ul. Świętego Pawła 36,False,48688771526,krz.le@gmail.com
1430,1430,Mirosław,Ćwikła,M,81050234230,Swarzędz,ul. Poleska 7,False,48847025940,d3g6m58420iwoyj@gmail.com
1431,1431,Anna,Szymańska,K,80091143057,Kędzierzyn-Koźle,ul. Zbożowa 13,False,48067130453,ann_szy80@interia.pl


In [78]:
pd.unique(clients_df["id"])

array([   0,    1,    2, ..., 1430, 1431, 1432])

In [79]:
clients_with_lc = clients_df[clients_df["karta_lojalnościowa"]]["id"].values

loyalty_card_dates = [() for i in range(len(clients_with_lc))]
for (i, client) in enumerate(clients_with_lc):
    loyalty_card_dates[i] = client, dates[np.where(client_ids == client)[0][2]]


In [80]:
clients_df

,id,imię,nazwisko,płeć,PESEL,miasto,adres,karta_lojalnościowa,telefon,email
0,0,Robert,Czopek,M,61111187231,Iłowa,ul. Piotra Skargi 16,False,48851178913,robert.czopek@kuba.pl
1,1,Lena,Skwarczek,K,60062446615,Wrocław,ul. Michała Poleskiego 28/47,False,48448167425,u01q1a8q1r2x0m52@opayq.com
2,2,Ewelina,Jasińska,K,76081037596,Drawno,ul. Bolesława Krzywoustego 70/30,False,48423071263,ejasinska76@gmail.com
3,3,Dawid,Lisewski,M,60092784275,Wrocław,ul. Pocztowa 60/15,False,48149667875,dlisewski60@gmail.com
4,4,Danuta,Pawlik,K,94051955617,Wrocław,ul. Wiosenna 27,False,48783613153,dan-paw@gmail.com
...,...,...,...,...,...,...,...,...,...,...
1428,1428,Ewa,Lutostańska,K,69011606838,Wrocław,ul. Ks. Jerzego Popiełuszki 69,False,48635885839,l3y6ksv3@opayq.com
1429,1429,Krzysztof,Lempaszak,M,97020931111,Piastów,ul. Świętego Pawła 36,False,48688771526,krz.le@gmail.com
1430,1430,Mirosław,Ćwikła,M,81050234230,Swarzędz,ul. Poleska 7,False,48847025940,d3g6m58420iwoyj@gmail.com
1431,1431,Anna,Szymańska,K,80091143057,Kędzierzyn-Koźle,ul. Zbożowa 13,False,48067130453,ann_szy80@interia.pl


In [81]:
clients_df.to_csv("outputs/klienci.csv", index=False)

In [82]:
np.sum(clients_df['karta_lojalnościowa'])

np.int64(77)

In [83]:
np.sum(pd.isna(clients_df["PESEL"]))

np.int64(220)

# auta

In [84]:
rng = np.random.default_rng()

data_car, data_bike = get_car_data()
make_list_car, model_list_car, car_weights = get_car_weights()
make_list_bike, model_list_bike, bike_weights = get_bike_weights()

In [85]:
def create_car(i):

    is_car = rng.random() < 0.9
    colors = np.array(('czarny', 'biały', 'szary', 'srebrny', 'niebieski', 'czerwony', 'brązowy', 'złoty', 'żółty', 'inny'))
    colors_weights = np.array((0.22, 0.19, 0.18, 0.15, 0.10, 0.09, 0.03, 0.02, 0.01, 0.01))

    if is_car:
        index = rng.choice(len(make_list_car), p=car_weights)
        make, model = make_list_car[index], model_list_car[index]
        weights = data_car[(data_car['make'] == make) & (data_car['model'] == model)]
        body = np.array(('SUV', 'Hatchback', 'Sedan', 'Kombi', 'Coupe', 'Van', 'Minivan', 'Kabriolet'))
        body_weights = np.array((0.25, 0.25, 0.23, 0.17, 0.06, 0.03, 0.009, 0.001))
    else:
        index = rng.choice(len(make_list_bike), p=bike_weights)
        make, model = make_list_bike[index], model_list_bike[index]
        weights = False
        body = False
        body_weights = False

    wheels = 4 if is_car else 2
    year = get_year()
    gear = get_gear(weights, is_car)
    fuel = get_fuel(weights, is_car)
    post_accident = get_post_accident()
    hp = get_hp(weights, is_car)
    color = get_color(colors, colors_weights)
    body_type = get_body_type(body, body_weights, is_car)
    engine_capacity = get_engine_capacity(hp, is_car)
    doors = get_doors(is_car)
    seats = get_num_of_seats(is_car)
    available = i+1 in our_cars
    
    return [i+1, make, model, wheels, year, body_type, post_accident, color, fuel, engine_capacity, hp, gear, doors, seats, available]


def create_car_table(n):
    
    df = pd.DataFrame(columns=['id', 'marka', 'model', 'liczba_kół', 'rok_produkcji', 'typ_nadwozia', 'czy_powypadkowy', 'kolor', 'typ_paliwa', 'pojemność_silnika', 'moc', 'skrzynia_biegów', 'liczba_drzwi', 'liczba_miejsc', 'czy_dostępny'])
    for i in range(n):
        row = create_car(i)
        df.loc[i] = row
    return df


In [86]:
df_samochody = create_car_table(max(car_ids))

In [87]:
df_samochody

,id,marka,model,liczba_kół,rok_produkcji,typ_nadwozia,czy_powypadkowy,kolor,typ_paliwa,pojemność_silnika,moc,skrzynia_biegów,liczba_drzwi,liczba_miejsc,czy_dostępny
0,1,Fiat,500C,4,2014,Minivan,False,czarny,Benzyna,1.3,86,Manualna,5,5,False
1,2,Honda,CBR 150,2,1997,,False,biały,Benzyna,0.4,57,Manualna,0,1,False
2,3,Renault,Espace,4,2005,Hatchback,True,szary,Diesel,2.8,173,Automatyczna,5,5,False
3,4,SEAT,Leon,4,2010,Kombi,False,biały,Benzyna,2.1,131,Manualna,5,2,False
4,5,Fiat,500,4,2011,Coupe,False,srebrny,Benzyna,0.9,69,Manualna,5,5,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1068,1069,Mercedes-Benz,C 220,4,2004,Hatchback,True,szary,Diesel,2.7,170,Manualna,5,2,True
1069,1070,Ford,Transit,4,2009,Sedan,False,srebrny,Diesel,2.6,155,Manualna,5,2,True
1070,1071,Toyota,Aygo,4,2003,Sedan,False,srebrny,Benzyna,1.1,72,Automatyczna,3,5,True
1071,1072,Opel,Astra,4,2011,Sedan,False,złoty,Benzyna,1.9,110,Manualna,3,2,True


In [88]:
df_samochody.to_csv("outputs/samochody.csv", index=False)

In [89]:
pd.unique(df_samochody["skrzynia_biegów"])

array(['Manualna', 'Automatyczna'], dtype=object)

# Usługi

In [90]:
service_employee = rng.choice(employees_df[employees_df['stanowisko'] == 'mechanik']['id'], len(car_ids_service))
service_ids = np.arange(len(transaction_dates_service)) + 1

df_services = pd.DataFrame()
df_services['id'] = service_ids
df_services['id_samochodu'] = car_ids_service
df_services['id_klienta'] = client_ids_service
df_services['id_pracownika'] = service_employee
df_services['data_zaksięgowania'] = transaction_dates_service

# Zakup samochodów

In [91]:
def car_purchase_price(hp, yr_bought, yr_prod, post_acc):
    return np.round(hp * 1000 / np.sqrt(int(yr_bought) - int(yr_prod) + 1) * (0.75 if post_acc else 1) * rng.uniform(0.8, 1.2), -2)

In [92]:
buy_employee = rng.choice(employees_df[employees_df['stanowisko'] == 'sprzedawca']['id'], len(car_ids_buy))
buy_ids = np.arange(len(transaction_dates_buy)) + 1

df_buy = pd.DataFrame()
df_buy['id'] = buy_ids
df_buy['id_samochodu'] = car_ids_buy
df_buy['id_klienta'] = client_ids_buy
df_buy['id_pracownika'] = buy_employee
df_buy['data_zaksięgowania'] = transaction_dates_buy

In [93]:

service_date_fix = pd.merge(df_samochody[['id', 'rok_produkcji']], df_services[['id_samochodu', 'data_zaksięgowania']], how='right', left_on='id', right_on='id_samochodu')
service_date_fix['data_zaksięgowania'] = service_date_fix['data_zaksięgowania'].str[:4]

buy_date_fix = pd.merge(df_samochody[['id', 'rok_produkcji']], df_buy[['id_samochodu', 'data_zaksięgowania']], how='right', left_on='id', right_on='id_samochodu')
buy_date_fix['data_zaksięgowania'] = buy_date_fix['data_zaksięgowania'].str[:4]

for i in range(len(buy_date_fix)):
    if str(buy_date_fix['rok_produkcji'].loc()[i]) >= str(buy_date_fix['data_zaksięgowania'].loc()[i]):
        car_index = buy_date_fix.loc[i, 'id_samochodu']
        df_samochody.loc[car_index-1, 'rok_produkcji'] = str(int(df_samochody.loc[car_index-1, 'rok_produkcji']) - 10)

for i in range(len(service_date_fix)):
    if str(service_date_fix['rok_produkcji'].loc()[i]) >= str(service_date_fix['data_zaksięgowania'].loc()[i]):
        car_index = service_date_fix.loc[i, 'id_samochodu']
        df_samochody.loc[car_index-1, 'rok_produkcji'] = str(int(df_samochody.loc[car_index-1, 'rok_produkcji']) - 10)

C:\Users\olkar\AppData\Local\Temp\ipykernel_40304\3526569416.py:10: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '2004' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df_samochody.loc[car_index-1, 'rok_produkcji'] = str(int(df_samochody.loc[car_index-1, 'rok_produkcji']) - 10)


In [94]:
purchase_prices = []
for j, i in enumerate(car_ids_buy):
    row = df_samochody.loc()[i-1]
    year_bought = transaction_dates_buy[j][:4]
    purchase_prices.append(car_purchase_price(row['moc'], year_bought, row['rok_produkcji'], row['czy_powypadkowy']))

df_buy['cena'] = purchase_prices

In [95]:
df_buy

,id,id_samochodu,id_klienta,id_pracownika,data_zaksięgowania,cena
0,1,1,1,6,2014-01-02,27900.0
1,2,2,2,6,2014-01-03,13600.0
2,3,3,3,7,2014-01-05,38200.0
3,4,4,4,3,2014-01-05,69800.0
4,5,5,5,7,2014-01-07,40200.0
...,...,...,...,...,...,...
747,748,1065,136,3,2017-12-23,31300.0
748,749,1069,1428,3,2017-12-28,38800.0
749,750,1070,1429,6,2017-12-28,49200.0
750,751,1071,217,2,2017-12-30,18100.0


In [96]:
df_buy.to_csv("outputs/zakup_samochodu.csv", index=False)

# Sprzedaż samochodów

In [97]:
sell_employee = rng.choice(employees_df[employees_df['stanowisko'] == 'sprzedawca']['id'], len(car_ids_sell))
sell_ids = np.arange(len(transaction_dates_sell)) + 1

df_sell = pd.DataFrame()
df_sell['id'] = sell_ids
df_sell['id_samochodu'] = car_ids_sell
df_sell['id_klienta'] = client_ids_sell
df_sell['id_pracownika'] = sell_employee
df_sell['data_zaksięgowania'] = transaction_dates_sell

In [98]:
loyalty_card_customer_ids = [el[0] for el in loyalty_card_dates]
loyalty_card_dates_sep = [el[1] for el in loyalty_card_dates]

In [99]:
sell_prices = []
for i in car_ids_sell:
    b_price = df_buy[df_buy['id_samochodu'] == i]['cena'].values[0]
    sell_prices.append(np.round(b_price * rng.uniform(1.2, 1.5), -2))

for i in range(len(sell_ids)):
    customer_id = df_sell.loc()[i]['id_klienta']
    if clients_df[clients_df['id'] == customer_id]['karta_lojalnościowa'].bool() and customer_id in loyalty_card_customer_ids:
        date_index = loyalty_card_customer_ids.index(customer_id)
        if loyalty_card_dates_sep[date_index] < df_sell.loc()[i]['data_zaksięgowania']:
            sell_prices[i] *= 0.97
            #print(i)

sell_installments = []
for i in range(len(sell_ids)):
    if sell_prices[i] > 80000:
        inst_prob = [0.1, 0.2, 0.3, 0.4]
    else:
        inst_prob = [0.4, 0.3, 0.2, 0.1]
    installments = rng.choice([1, 6, 12, 18], p=inst_prob)
    if installments == 6:
        sell_prices[i] *= 1.02
    elif installments == 12:
        sell_prices[i] *= 1.04
    elif installments == 18:
        sell_prices[i] *= 1.06
    sell_installments.append(installments)
#print(sell_installments)

df_sell['cena'] = sell_prices
df_sell['liczba_rat'] = sell_installments
df_sell = df_sell[['id', 'id_samochodu', 'id_klienta', 'id_pracownika', 'cena', 'liczba_rat', 'data_zaksięgowania']]

C:\Users\olkar\AppData\Local\Temp\ipykernel_40304\1822833189.py:8: FutureWarning: Series.bool is now deprecated and will be removed in future version of pandas
  if clients_df[clients_df['id'] == customer_id]['karta_lojalnościowa'].bool() and customer_id in loyalty_card_customer_ids:


In [100]:
df_sell

,id,id_samochodu,id_klienta,id_pracownika,cena,liczba_rat,data_zaksięgowania
0,1,10,21,3,55794.0,6,2014-01-15
1,2,3,22,7,55500.0,1,2014-01-15
2,3,6,23,7,31900.0,1,2014-01-15
3,4,19,24,2,25398.0,6,2014-01-17
4,5,2,25,2,18768.0,6,2014-01-17
...,...,...,...,...,...,...,...
743,744,1065,1293,6,40086.0,6,2017-12-24
744,745,1063,1100,7,29536.0,12,2017-12-24
745,746,1061,1430,3,40494.0,6,2017-12-29
746,747,1064,1315,3,89252.0,18,2017-12-29


In [101]:
np.sum(df_sell["cena"] == float('inf')) # wychodzą inf wiec trzeba poprawić

np.int64(0)

In [102]:
df_sell.to_csv("outputs/sprzedaż_samochodu.csv", index=False)

# Użyte części

In [103]:
data_parts = get_data_parts()
data_parts_filtered = data_parts[data_parts['motor'] == True]

def get_parts_indexes(car_id):
    if df_samochody.loc[car_id-1].values[3] == 2:
        parts_set = data_parts_filtered[['id','część']]
    else:
        parts_set = data_parts[['id','część']]
    
    parts_indexes = parts_set['id']
    num_parts = rng.choice([1, 2, 3], p=[0.6, 0.3, 0.1])
    indexes = rng.choice(parts_indexes, size=num_parts)
    return indexes

all_car_parts_indexes = [(i, get_parts_indexes(i)) for i in car_ids_service]
print(all_car_parts_indexes)

[(np.int64(22), array([55, 42])), (np.int64(23), array([31])), (np.int64(26), array([10, 28])), (np.int64(27), array([53])), (np.int64(29), array([62, 45])), (np.int64(30), array([39, 34])), (np.int64(31), array([33, 35])), (np.int64(33), array([63])), (np.int64(39), array([14])), (np.int64(43), array([19, 54])), (np.int64(46), array([18])), (np.int64(49), array([51])), (np.int64(50), array([60, 51])), (np.int64(51), array([23])), (np.int64(56), array([41, 34])), (np.int64(57), array([61])), (np.int64(63), array([36])), (np.int64(64), array([37, 13])), (np.int64(68), array([57, 51])), (np.int64(71), array([20, 27])), (np.int64(72), array([48])), (np.int64(75), array([62, 51])), (np.int64(76), array([8, 7])), (np.int64(79), array([15, 61,  8])), (np.int64(83), array([60])), (np.int64(86), array([9])), (np.int64(87), array([48])), (np.int64(91), array([49])), (np.int64(92), array([46])), (np.int64(93), array([55])), (np.int64(100), array([5])), (np.int64(99), array([54, 52])), (np.int64(

In [104]:
#df_samochody.loc()[1] - samochód o indeksie 0

def create_part(id, service_id, car):
    part = data_parts.loc()[id]
    if type(part['atrybuty']) != float:
        attributes = part['atrybuty'].split()
        for i in range(len(attributes)):
            if attributes[i] == 'marka':
                attributes[i] = car['marka']
            if attributes[i] == 'model':
                attributes[i] = car['model']
            if attributes[i] == 'kolor':
                attributes[i] = car['kolor']
            if attributes[i] == 'rozmiar':
                attributes[i] = 'R' + str(rng.choice([15, 16, 17]))
            if attributes[i] == 'material':
                attributes[i] = rng.choice(['skóra', 'materiał'], p=[0.2, 0.8])
            if attributes[i] == 'pojemnosc':
                attributes[i] = str(car['pojemność_silnika'])
            if attributes[i] == 'skrzynia':
                attributes[i] = str(car['skrzynia_biegów'])
        price = int(np.floor(part['cena']*car['pojemność_silnika']/1.8 * rng.uniform(0.8, 1.2)))
    else:
        attributes = ''
        price = part['cena']
    quantity = part['maxcount']
    return part['część'], ' '.join(attributes), price, service_id, quantity

parts = []
for service_id, element in enumerate(all_car_parts_indexes):
    car_id, parts_id = element
    car = df_samochody.loc()[car_id-1]
    for part_id in parts_id:
        parts.append(create_part(part_id, service_id+1, car))

parts = np.array(parts)
parts_df = pd.DataFrame(parts, columns=['nazwa', 'atrybuty', 'cena', 'id_usługi', 'ilość']).drop_duplicates(['nazwa', 'atrybuty'])
parts_df['id'] = np.arange(1, len(parts_df)+1)
parts_df = parts_df[['id', 'nazwa', 'atrybuty', 'cena', 'id_usługi', 'ilość']]

parts = pd.DataFrame(parts, columns=['nazwa', 'atrybut', 'cena', 'id_usługi', 'ilość'])

In [105]:
parts

,nazwa,atrybut,cena,id_usługi,ilość
0,Wał sterujący,,150,1,1
1,Chłodnica,Kia Sportage,738,1,1
2,Zagłówek,Mazda 3,22,2,2
3,Drzwi lewe przednie,Volkswagen Tiguan niebieski,519,3,1
4,Bezpiecznik,,10,3,1
...,...,...,...,...,...
770,Skrzynia biegów,BMW X2 Automatyczna,3357,522,1
771,Rura wydechowa,,100,522,1
772,Wtrysk paliwa,Bajaj Platina 100,76,523,1
773,Rura wydechowa,,100,523,1


# Wyposażenie

In [106]:
eq_len = len(parts_df)
equipment_id = parts_df['id']
equipment_name = parts_df['nazwa']
equipment_attr = parts_df['atrybuty']
equipment_price = parts_df['cena']

df_equipment = pd.DataFrame()
df_equipment['id'] = equipment_id
df_equipment['nazwa'] = equipment_name
df_equipment['atrybut'] = equipment_attr
df_equipment['cena_części'] = equipment_price.astype(float)

In [107]:
df_equipment

,id,nazwa,atrybut,cena_części
0,1,Wał sterujący,,150.0
1,2,Chłodnica,Kia Sportage,738.0
2,3,Zagłówek,Mazda 3,22.0
3,4,Drzwi lewe przednie,Volkswagen Tiguan niebieski,519.0
4,5,Bezpiecznik,,10.0
...,...,...,...,...
765,525,Drzwi prawe tylne,Jeep Compass czerwony,632.0
766,526,Wewnętrzna klamka drzwi,Jeep Compass,16.0
769,527,Zagłówek,Opel Zafira,28.0
770,528,Skrzynia biegów,BMW X2 Automatyczna,3357.0


# Użyte części - uzupełnienie

In [108]:
used_parts_df = parts.merge(df_equipment, on=['nazwa', 'atrybut'])
used_parts_df.rename(columns={'id': 'id_części'}, inplace=True)
used_parts_df['ilość'] = pd.to_numeric(used_parts_df['ilość'])
ids = np.arange(len(used_parts_df))+1
used_parts_df['id'] = ids
used_parts_df['cena'] = pd.to_numeric(used_parts_df['cena'])

for i in range(len(used_parts_df)):
    used_parts_df.loc()[i, 'cena'] *= rng.uniform(0.9, 1.1)
    used_parts_df.loc()[i, 'cena'] = np.round(used_parts_df.loc()[i, 'cena'], 0)

used_parts_df = used_parts_df[['id', 'id_usługi', 'id_części', 'ilość', 'cena']]

C:\Users\olkar\AppData\Local\Temp\ipykernel_40304\1878107147.py:9: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '162.8986012173062' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  used_parts_df.loc()[i, 'cena'] *= rng.uniform(0.9, 1.1)


In [109]:
used_parts_df

,id,id_usługi,id_części,ilość,cena
0,1,1,1,1,163.0
1,2,1,2,1,761.0
2,3,2,3,2,21.0
3,4,3,4,1,543.0
4,5,3,5,1,10.0
...,...,...,...,...,...
770,771,522,528,1,3688.0
771,772,522,8,1,103.0
772,773,523,529,1,73.0
773,774,523,8,1,93.0


In [110]:
used_parts_df.to_csv("outputs/użyte_części.csv", index=False)

# Zakup części

In [111]:
# def get_service_date(part_id):
    
#     index_used_part = used_parts_df[used_parts_df['id_części'] == part_id].index[0]
#     service_date = df_services[df_services['id'] == index_used_part]['data_zaksięgowania']
#     return service_date
    

# buy_parts_quantity = used_parts_df.groupby(['id_części'])['ilość'].sum().reset_index()
# parts_purchase_pd = pd.DataFrame(columns=['id', 'id_części', 'koszt', 'ilość'])

# for i in range(len(buy_parts_quantity)):
#     part_id = buy_parts_quantity['id_części'][i]
#     quantity = buy_parts_quantity['ilość'][i]
#     price = df_equipment['cena_części'][df_equipment['id'] == part_id].to_numpy()
#     total_cost = np.floor(price * rng.uniform(0.75, 0.95))[0]
#     service_date = get_service_date(part_id)
    
#     row = [int(i+1), int(part_id), total_cost, int(quantity)]
#     parts_purchase_pd.loc[i] = row


In [112]:
# parts_purchase_pd

In [113]:
# parts_purchase_pd.to_csv("outputs/zakup_części.csv", index=False)

In [114]:
df_equipment = df_equipment[['id', 'nazwa', 'atrybut']]

In [115]:
df_equipment.to_csv("outputs/wyposażenie.csv", index=False)

# Usługi uzupełnienie

In [116]:
service_prices = np.zeros(len(service_ids))
for i in range(len(parts)):
    row = parts.loc()[i]
    service_index = int(row['id_usługi'])
    service_prices[service_index-1] += float(row['cena']) * int(row['ilość'])

for i in range(len(df_services)):
    if df_services.loc()[i]['id_klienta'] == 0:
        df_services.loc[i, 'cena'] = 0
    else:
        df_services.loc[i, 'cena'] = np.round(service_prices[i] * (rng.uniform(1.25, 1.7)), -1).astype(int)

In [117]:
df_services

,id,id_samochodu,id_klienta,id_pracownika,data_zaksięgowania,cena
0,1,22,43,8,2014-01-30,1120.0
1,2,23,44,8,2014-01-30,70.0
2,3,26,49,9,2014-02-02,830.0
3,4,27,50,8,2014-02-02,680.0
4,5,29,53,4,2014-02-07,150.0
...,...,...,...,...,...,...
519,520,1066,1426,9,2017-12-24,1730.0
520,521,1067,1427,8,2017-12-26,90.0
521,522,1068,914,8,2017-12-27,4700.0
522,523,262,419,9,2017-12-30,260.0


In [118]:
df_services.to_csv("outputs/usługi.csv", index=False)